# Installing and importing the dependencies

In [4]:
!pip install opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow

# Key Points using media pipes

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [7]:
cap.release()
cv2.destroyAllWindows()

# Extracting the key points

In [8]:
len(results.pose_landmarks.landmark)

33

In [9]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [10]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [11]:
lh

array([ 7.93887198e-01,  8.39927137e-01,  3.03519869e-07,  7.47450888e-01,
        8.10416341e-01, -1.90511420e-02,  7.03222275e-01,  7.59046078e-01,
       -2.92247217e-02,  6.67830348e-01,  7.14722872e-01, -3.81533876e-02,
        6.39782429e-01,  6.80067420e-01, -4.66625281e-02,  7.30456650e-01,
        6.28897130e-01, -1.47098964e-02,  7.09133804e-01,  5.45180678e-01,
       -2.79654395e-02,  6.96832001e-01,  4.93692338e-01, -3.93457003e-02,
        6.88132346e-01,  4.48350072e-01, -4.84100245e-02,  7.60626793e-01,
        6.13119483e-01, -1.68344565e-02,  7.46185064e-01,  5.21145105e-01,
       -2.78632063e-02,  7.37348974e-01,  4.61489439e-01, -3.85412239e-02,
        7.29599893e-01,  4.11503941e-01, -4.70456704e-02,  7.91560590e-01,
        6.19293988e-01, -2.20718496e-02,  7.92813778e-01,  5.34219086e-01,
       -3.65654454e-02,  7.92735100e-01,  4.78599966e-01, -4.77746576e-02,
        7.90600955e-01,  4.29127038e-01, -5.58535829e-02,  8.21294785e-01,
        6.42847538e-01, -

In [12]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [13]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [14]:
extract_keypoints(results).shape

(1662,)

In [15]:
result_test = extract_keypoints(results)

In [16]:
np.save('0',result_test)

In [17]:
np.load('0.npy')

array([ 0.48744637,  0.70218223, -0.9371444 , ...,  0.        ,
        0.        ,  0.        ])

# Setting up folders for collection

In [18]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [19]:
# hello
## 0
## 1
## 2
## ...
## 29
# thanks

# I love you

In [20]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collecting keypoint values for testing and training

In [21]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# Preprocess data and create labels and features

In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [23]:
label_map = {label:num for num, label in enumerate(actions)}

In [24]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [25]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [27]:
np.array(sequences).shape

(90, 30, 1662)

In [28]:
np.array(labels).shape

(90,)

In [29]:
X = np.array(sequences)

In [30]:
X.shape

(90, 30, 1662)

In [31]:
y = to_categorical(labels).astype(int)

In [32]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [39]:
y_test.shape

(5, 3)

# Build and train our LSTM neural network

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [41]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [42]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

2022-01-22 15:03:52.582807: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [43]:
res = [.7, 0.2, 0.1]

In [44]:
actions[np.argmax(res)]

'hello'

In [45]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [46]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 7s 213ms/step - loss: 1.3092 - categorical_accuracy: 0.3529
Epoch 2/2000
3/3 [==============================] - 0s 128ms/step - loss: 20.5313 - categorical_accuracy: 0.3529
Epoch 3/2000
3/3 [==============================] - 0s 128ms/step - loss: 7.4756 - categorical_accuracy: 0.2941
Epoch 4/2000
3/3 [==============================] - 1s 192ms/step - loss: 11.2404 - categorical_accuracy: 0.3765
Epoch 5/2000
3/3 [==============================] - 0s 159ms/step - loss: 38.6723 - categorical_accuracy: 0.3059
Epoch 6/2000
3/3 [==============================] - 1s 223ms/step - loss: 8.3190 - categorical_accuracy: 0.2824
Epoch 7/2000
3/3 [==============================] - 0s 165ms/step - loss: 4.3824 - categorical_accuracy: 0.4588
Epoch 8/2000
3/3 [==============================] - 1s 233ms/step - loss: 3.3112 - categorical_accuracy: 0.3059
Epoch 9/2000
3/3 [==============================] - 1s 189ms/step - loss: 2.1477 - categorical_accura

3/3 [==============================] - 1s 213ms/step - loss: 0.7751 - categorical_accuracy: 0.5765
Epoch 74/2000
3/3 [==============================] - 1s 204ms/step - loss: 0.7462 - categorical_accuracy: 0.6471
Epoch 75/2000
3/3 [==============================] - 1s 212ms/step - loss: 0.7558 - categorical_accuracy: 0.6588
Epoch 76/2000
3/3 [==============================] - 1s 234ms/step - loss: 0.6743 - categorical_accuracy: 0.6706
Epoch 77/2000
3/3 [==============================] - 1s 164ms/step - loss: 0.7240 - categorical_accuracy: 0.6471
Epoch 78/2000
3/3 [==============================] - 1s 168ms/step - loss: 0.7435 - categorical_accuracy: 0.6588
Epoch 79/2000
3/3 [==============================] - 1s 201ms/step - loss: 0.6882 - categorical_accuracy: 0.6471
Epoch 80/2000
3/3 [==============================] - 1s 203ms/step - loss: 0.6714 - categorical_accuracy: 0.6941
Epoch 81/2000
3/3 [==============================] - 1s 235ms/step - loss: 0.6584 - categorical_accuracy: 0.68

3/3 [==============================] - 0s 130ms/step - loss: 0.5315 - categorical_accuracy: 0.6824
Epoch 146/2000
3/3 [==============================] - 0s 125ms/step - loss: 0.4352 - categorical_accuracy: 0.8000
Epoch 147/2000
3/3 [==============================] - 0s 121ms/step - loss: 0.4407 - categorical_accuracy: 0.7882
Epoch 148/2000
3/3 [==============================] - 0s 123ms/step - loss: 0.4456 - categorical_accuracy: 0.7647
Epoch 149/2000
3/3 [==============================] - 0s 122ms/step - loss: 0.4216 - categorical_accuracy: 0.7647
Epoch 150/2000
3/3 [==============================] - 0s 123ms/step - loss: 0.3930 - categorical_accuracy: 0.8000
Epoch 151/2000
3/3 [==============================] - 0s 122ms/step - loss: 0.3696 - categorical_accuracy: 0.8471
Epoch 152/2000
3/3 [==============================] - 0s 128ms/step - loss: 0.3937 - categorical_accuracy: 0.8000
Epoch 153/2000
3/3 [==============================] - 0s 133ms/step - loss: 0.3527 - categorical_accura

Epoch 217/2000
3/3 [==============================] - 0s 119ms/step - loss: 0.4061 - categorical_accuracy: 0.8118
Epoch 218/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.4264 - categorical_accuracy: 0.7647
Epoch 219/2000
3/3 [==============================] - 0s 116ms/step - loss: 0.4337 - categorical_accuracy: 0.7412
Epoch 220/2000
3/3 [==============================] - 0s 118ms/step - loss: 0.3824 - categorical_accuracy: 0.7882
Epoch 221/2000
3/3 [==============================] - 0s 120ms/step - loss: 0.3866 - categorical_accuracy: 0.8235
Epoch 222/2000
3/3 [==============================] - 0s 131ms/step - loss: 0.4184 - categorical_accuracy: 0.7529
Epoch 223/2000
3/3 [==============================] - 0s 121ms/step - loss: 0.3772 - categorical_accuracy: 0.8000
Epoch 224/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.3641 - categorical_accuracy: 0.7882
Epoch 225/2000
3/3 [==============================] - 0s 124ms/step - loss: 0.3716 - cat

3/3 [==============================] - 0s 129ms/step - loss: 1.1441 - categorical_accuracy: 0.5176
Epoch 289/2000
3/3 [==============================] - 0s 120ms/step - loss: 0.8112 - categorical_accuracy: 0.6471
Epoch 290/2000
3/3 [==============================] - 0s 131ms/step - loss: 0.7130 - categorical_accuracy: 0.6471
Epoch 291/2000
3/3 [==============================] - 0s 113ms/step - loss: 0.7145 - categorical_accuracy: 0.7529
Epoch 292/2000
3/3 [==============================] - 0s 119ms/step - loss: 0.4537 - categorical_accuracy: 0.8000
Epoch 293/2000
3/3 [==============================] - 1s 189ms/step - loss: 0.3857 - categorical_accuracy: 0.8353
Epoch 294/2000
3/3 [==============================] - 0s 133ms/step - loss: 0.4087 - categorical_accuracy: 0.7765
Epoch 295/2000
3/3 [==============================] - 0s 162ms/step - loss: 0.4056 - categorical_accuracy: 0.8353
Epoch 296/2000
3/3 [==============================] - 1s 164ms/step - loss: 0.3932 - categorical_accura

...

Epoch 932/2000
3/3 [==============================] - 1s 193ms/step - loss: 0.3631 - categorical_accuracy: 0.8118
Epoch 933/2000
3/3 [==============================] - 1s 189ms/step - loss: 0.3534 - categorical_accuracy: 0.8353
Epoch 934/2000
3/3 [==============================] - 1s 211ms/step - loss: 0.3425 - categorical_accuracy: 0.8471
Epoch 935/2000
3/3 [==============================] - 1s 197ms/step - loss: 0.3307 - categorical_accuracy: 0.8471
Epoch 936/2000
3/3 [==============================] - 1s 163ms/step - loss: 0.4114 - categorical_accuracy: 0.7765
Epoch 937/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.4058 - categorical_accuracy: 0.7882
Epoch 938/2000
3/3 [==============================] - 0s 121ms/step - loss: 0.3942 - categorical_accuracy: 0.7882
Epoch 939/2000
3/3 [==============================] - 0s 175ms/step - loss: 0.4014 - categorical_accuracy: 0.7765
Epoch 940/2000
3/3 [==============================] - 0s 138ms/step - loss: 0.3423 - cat

3/3 [==============================] - 1s 220ms/step - loss: 0.3519 - categorical_accuracy: 0.8000
Epoch 1004/2000
3/3 [==============================] - 0s 138ms/step - loss: 0.3085 - categorical_accuracy: 0.8824
Epoch 1005/2000
3/3 [==============================] - 0s 138ms/step - loss: 0.3604 - categorical_accuracy: 0.8235
Epoch 1006/2000
3/3 [==============================] - 0s 139ms/step - loss: 0.3770 - categorical_accuracy: 0.8000
Epoch 1007/2000
3/3 [==============================] - 0s 133ms/step - loss: 0.4139 - categorical_accuracy: 0.7412
Epoch 1008/2000
3/3 [==============================] - 1s 160ms/step - loss: 0.3681 - categorical_accuracy: 0.8000
Epoch 1009/2000
3/3 [==============================] - 0s 136ms/step - loss: 0.4051 - categorical_accuracy: 0.7647
Epoch 1010/2000
3/3 [==============================] - 0s 132ms/step - loss: 0.3166 - categorical_accuracy: 0.8706
Epoch 1011/2000
3/3 [==============================] - 0s 140ms/step - loss: 0.2995 - categorica

3/3 [==============================] - 0s 147ms/step - loss: 0.5453 - categorical_accuracy: 0.7412
Epoch 1075/2000
3/3 [==============================] - 1s 183ms/step - loss: 0.4000 - categorical_accuracy: 0.8000
Epoch 1076/2000
3/3 [==============================] - 0s 153ms/step - loss: 0.3325 - categorical_accuracy: 0.8471
Epoch 1077/2000
3/3 [==============================] - 0s 132ms/step - loss: 0.3196 - categorical_accuracy: 0.9059
Epoch 1078/2000
3/3 [==============================] - 0s 155ms/step - loss: 0.2997 - categorical_accuracy: 0.8588
Epoch 1079/2000
3/3 [==============================] - 0s 137ms/step - loss: 0.2784 - categorical_accuracy: 0.8941
Epoch 1080/2000
3/3 [==============================] - 0s 135ms/step - loss: 0.2738 - categorical_accuracy: 0.8824
Epoch 1081/2000
3/3 [==============================] - 0s 142ms/step - loss: 0.2915 - categorical_accuracy: 0.8706
Epoch 1082/2000
3/3 [==============================] - 1s 196ms/step - loss: 0.3483 - categorica

3/3 [==============================] - 1s 278ms/step - loss: 0.7828 - categorical_accuracy: 0.7647
Epoch 1146/2000
3/3 [==============================] - 1s 193ms/step - loss: 0.6627 - categorical_accuracy: 0.6471
Epoch 1147/2000
3/3 [==============================] - 1s 213ms/step - loss: 0.4116 - categorical_accuracy: 0.7882
Epoch 1148/2000
3/3 [==============================] - 1s 200ms/step - loss: 0.5934 - categorical_accuracy: 0.6824
Epoch 1149/2000
3/3 [==============================] - 1s 156ms/step - loss: 0.4211 - categorical_accuracy: 0.7882
Epoch 1150/2000
3/3 [==============================] - 1s 149ms/step - loss: 0.5348 - categorical_accuracy: 0.6706
Epoch 1151/2000
3/3 [==============================] - 0s 134ms/step - loss: 0.4579 - categorical_accuracy: 0.7529
Epoch 1152/2000
3/3 [==============================] - 0s 139ms/step - loss: 0.4621 - categorical_accuracy: 0.7059
Epoch 1153/2000
3/3 [==============================] - 1s 167ms/step - loss: 0.4455 - categorica

KeyboardInterrupt: 

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# Make Predictions

In [70]:
res = model.predict(X_test)

In [71]:
res

array([[6.8135554e-01, 5.0145322e-03, 3.1363001e-01],
       [2.1927401e-03, 8.2804054e-06, 9.9779898e-01],
       [6.9446467e-02, 6.9222930e-05, 9.3048429e-01],
       [7.6997196e-03, 8.7684754e-07, 9.9229938e-01],
       [5.8435470e-01, 2.7755455e-03, 4.1286978e-01]], dtype=float32)

In [82]:
actions[np.argmax(res[4])]

'hello'

In [83]:
actions[np.argmax(y_test[4])]

'hello'

# Save Weights

In [84]:
model.save('action.h5')

# Evaluation using confusion matrix and accuracy

In [85]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [86]:
yhat = model.predict(X_test)

In [87]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [88]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[2, 0],
        [0, 3]]])

In [89]:
accuracy_score(ytrue, yhat)

1.0

# Testing in real time 

In [91]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [95]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.soluti

iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe

KeyboardInterrupt: 

In [96]:
cap.release()
cv2.destroyAllWindows()